In [1]:
import findspark
findspark.init()
from pyspark import SparkContext

sc = SparkContext("local[10]","bruted_force")

In [2]:
import time

In [3]:
input_rdd = sc.textFile("./pagerank_data.txt")

In [4]:
BLOCK_SIZE = [16,8,4,2,1]
MAX_ITER_COUNT = 20
HYPER_PARAMETER = 0.85
time_result = []
CONVERGENCE_TOL = 0.0001

In [9]:
count = 0 

In [5]:
input_rdd = input_rdd.map(lambda x : [int(y) for y in x.split(' ')]).map(lambda x : (x[0],x[1:]))

In [10]:
time_start = time.time()
p_i = sc.broadcast([1/30000 for _ in range(30000)])
for j in range(20):
  count+=1
  key_pair = input_rdd.flatMap(lambda x:[ (i,p_i.value[x[0]] / len(x[1])) for i in x[1]])
  new_p_i = key_pair.reduceByKey(lambda x,y : x+y)

  post_process_p_i = new_p_i.map(lambda x : (x[0],HYPER_PARAMETER*x[1] + (1-HYPER_PARAMETER)/30000))


  temp_p_i = [(1-HYPER_PARAMETER)/30000 for _ in range(30000)]
  result = post_process_p_i.collect()

  for r in result:
    temp_p_i[r[0]] = r[1]

  err = sum([abs(p_i.value[i] - temp_p_i[i]) for i in range(30000)])
  if err < CONVERGENCE_TOL:
    break

  # print(err,temp_p_i)
  p_i = sc.broadcast(temp_p_i)

time_result.append(time.time()-time_start)

In [11]:
count

4